In [1]:
import urllib2
from bs4 import BeautifulSoup
import re
import pandas as pd
from tqdm import tqdm

In [2]:
newseum = "http://www.newseum.org/todaysfrontpages/?tfp_display=list&tfp_region=USA&tfp_sort_by=state"

In [180]:
page = urllib2.urlopen(newseum)
root = BeautifulSoup(page, 'html.parser')

In [184]:
res = root.find_all('a', {'class': 'tfp-list-link'})

In [185]:
names = [x.text.split('\n')[1] for x in res]

In [186]:
links = [x['href'] for x in res]

In [187]:
loc_name = [x.find('h6').text for x in res]

In [188]:
city_name = [x.split(',')[0] for x in loc_name]

In [189]:
state_name = [re.findall(', ([A-Za-z. ]*)', x)[0][:-4] for x in loc_name]

In [190]:
local_news = pd.DataFrame(zip(names, links, city_name, state_name), columns=['name', 'link', 'city', 'state'])

In [128]:
def getOuterLink(newseum_link):
    try:
        newseum_link = "http://www.newseum.org/" + newseum_link
        x = urllib2.urlopen(newseum_link)
        x = BeautifulSoup(x, 'html.parser')
        return x.find_all('span', {'class': 'fa fa-external-link'})[0].parent['href']
    except:
        return None

In [136]:
local_news.to_csv('./local_news.csv', encoding='utf-8')

In [140]:
sample = local_news.sample(100)

In [141]:
sample['outer_links'] = sample['link'].map(getOuterLink)

In [147]:
sample.to_pickle('sample.pkl')
sample.to_csv('sample.csv', encoding='utf-8')

In [250]:
def getHeadlines(link):
    try:
        page = urllib2.urlopen(link)
        root = BeautifulSoup(page, 'html.parser')
        MIN_HEADLINE_LEN = 5
        return [x.text.strip() for x in root.find_all('a') if len(x.text.strip().split(' ')) >= MIN_HEADLINE_LEN]
    except:
        return None

In [259]:
def getOpinions(link):
    try:
        page = urllib2.urlopen(link + '/opinion')
        root = BeautifulSoup(page, 'html.parser')
        MIN_HEADLINE_LEN = 5
        return [x.text.strip() for x in root.find_all('a') if len(x.text.strip().split(' ')) >= MIN_HEADLINE_LEN]
    except:
        return None

In [262]:
mini_sample = sample.sample(10)

In [263]:
mini_sample['headlines'] = mini_sample['outer_links'].map(getHeadlines)

In [264]:
mini_sample['opeds'] = mini_sample['outer_links'].map(getOpinions)